<a href="https://colab.research.google.com/github/govardhan-06/Eng-Mal-Translator/blob/main/Translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets -q
!pip install --upgrade accelerate -q
!pip uninstall -y transformers accelerate -q
!pip install transformers[torch] accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset

dataset= load_dataset("Govardhan-06/flores_eng_mal")
dataset

Generating train split:   0%|          | 0/1808 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/201 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence_eng_Latn', 'sentence_mal_Mlym'],
        num_rows: 1808
    })
    test: Dataset({
        features: ['sentence_eng_Latn', 'sentence_mal_Mlym'],
        num_rows: 201
    })
})

In [3]:
#Renaming test data to validation data
dataset['validation']=dataset.pop('test')

In [4]:
dataset['train'][1500]

{'sentence_eng_Latn': 'Since 1988, ballot boxes must be transparent so that voters and observers can witness that no envelopes are present at the start of the vote and that no envelopes are added except those of the duly counted and authorized voters.',
 'sentence_mal_Mlym': '1988 മുതൽ ബാലറ്റ് ബോക്സുകൾ സുതാര്യമാണ്, അതിലൂടെ വോട്ടർമാർക്കും നിരീക്ഷകർക്കും വോട്ടെടുപ്പിന്റെ തുടക്കത്തിൽ ഒരു എൻ\u200cവലപ്പുകളും ഇല്ലെന്നതും ശരിയായി എണ്ണിയിട്ടുള്ളതും അംഗീകൃതവുമായ വോട്ടർമാർക്കും ഒഴികെ എൻ\u200cവലപ്പുകളൊന്നും ചേർത്തിട്ടില്ലെന്നതും കൃത്യമായി കാണാനും നിരീക്ഷിക്കാനും സാധിക്കും.'}

In [5]:
from transformers import MarianMTModel, MarianTokenizer

model_ckpt="Helsinki-NLP/opus-mt-en-ml"
tokenizer = MarianTokenizer.from_pretrained(model_ckpt,return_tensors='pt')
model = MarianMTModel.from_pretrained(model_ckpt)

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/449k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/614k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/956k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/229M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [6]:
dataset['train']['sentence_eng_Latn'][0]

'Many buildings are quite beautiful to look at and the view from a tall building or from a cleverly-positioned window can be a beauty to behold.'

In [7]:
dataset['train']['sentence_mal_Mlym'][0]

'പല കെട്ടിടങ്ങളും കാണാൻ നല്ല ഭംഗിയാണ് കൂടാതെ ഉയരമുള്ള ഒരു കെട്ടിടത്തിൽ നിന്നോ അല്ലെങ്കിൽ ഉചിതമായി സ്ഥാനം നിശ്ചയിച്ച് പണിത ഒരു ജനലിലൂടെയോ കാണുന്ന കാഴ്ച്ച മനോഹരമായിരിക്കും.'

In [8]:
eng_sent= dataset['train']['sentence_eng_Latn'][0]
mal_sent= dataset['train']['sentence_mal_Mlym'][0]

model_input = tokenizer(eng_sent, text_target=mal_sent)
model_input

{'input_ids': [2186, 15180, 41, 6223, 2931, 8, 1034, 145, 6, 4, 1883, 52, 13, 13060, 3901, 121, 52, 13, 19575, 191, 20, 15930, 55, 8281, 137, 33, 13, 6070, 8, 2229, 2, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [788, 5675, 465, 1995, 432, 9884, 291, 1752, 3132, 381, 32, 5675, 109, 10460, 1060, 8863, 45, 4240, 4658, 25, 12024, 32, 1127, 180, 2091, 256, 3694, 7226, 389, 9572, 3213, 2, 0]}

In [9]:
dataset['train'].column_names

['sentence_eng_Latn', 'sentence_mal_Mlym']

In [25]:
dataset['train'][0]['sentence_eng_Latn']

'Many buildings are quite beautiful to look at and the view from a tall building or from a cleverly-positioned window can be a beauty to behold.'

In [10]:
def tokenize_function(examples):
  input= examples["sentence_eng_Latn"]
  target= examples["sentence_mal_Mlym"]
  model_input = tokenizer(input, text_target=target, truncation=True)
  return model_input

In [11]:
tokenized_inputs=dataset.map(tokenize_function,batched=True,remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/1808 [00:00<?, ? examples/s]

Map:   0%|          | 0/201 [00:00<?, ? examples/s]

### Fine-tuning

In [12]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)

In [13]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [14]:
batch = data_collator([tokenized_inputs["train"][i] for i in range(1, 3)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [15]:
batch['labels']

tensor([[ 1859,   205,  1184, 10500,   193,  1897,    99,   513,  1463,   160,
          9223, 15540,  7590,   308,   567,  2162,  7971,  9206,     2,     0,
          -100,  -100,  -100],
        [ 2896,    95,  2439,  2784,   705,   176,  1568,  6230,  6972,  3583,
          3739,  1454,  2959,  2519,  2467,  1133, 16000,    25,  1959,   270,
         14102,     2,     0]])

In [16]:
!pip install evaluate -q
!pip install sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.8 MB/s eta 0:00:00


In [17]:
import evaluate

metric = evaluate.load("sacrebleu")

In [18]:
import numpy as np
from datasets import load_metric
from transformers import EvalPrediction

# Load the BLEU metric
metric = load_metric("sacrebleu")

def compute_metrics(eval_preds: EvalPrediction):
    preds, labels = eval_preds

    # If preds is a tuple, extract the first element
    if isinstance(preds, tuple):
        preds = preds[0]

    # Ensure that preds is a numpy array
    preds = np.array(preds)

    # If preds are logits, get the token IDs by taking argmax
    if preds.ndim == 3:  # Assuming shape (batch_size, sequence_length, vocab_size)
        preds = np.argmax(preds, axis=-1)

    # Replace -100 in labels with the tokenizer pad token ID
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Decode the predictions and labels
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Post-process the decoded outputs
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    # SacreBLEU metric expects references to be a list of lists
    decoded_labels = [[label] for label in decoded_labels]

    # Compute the BLEU score
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)

    # Return the BLEU score
    return {"bleu": result["score"]}


<ipython-input-18-21d7d89731d5>:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


The repository for sacrebleu contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/sacrebleu.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [19]:
from huggingface_hub import notebook_login

notebook_login()

In [20]:
from transformers import Seq2SeqTrainingArguments

# Adjusting the dropout rate in the model configuration
model.config.dropout_rate = 0.1

# Training arguments
args = Seq2SeqTrainingArguments(
    output_dir=model_ckpt,
    evaluation_strategy="epoch",  # Evaluate after each epoch
    save_strategy="epoch",
    learning_rate=3e-5,  # Experiment with different values
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=60,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
    load_best_model_at_end=True,
    metric_for_best_model="bleu",  #BLEU for evaluation
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [21]:
from transformers import Seq2SeqTrainer

# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_inputs['train'],
    eval_dataset=tokenized_inputs['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,  # Define a function to compute BLEU or other metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,Bleu
1,No log,2.864270,4.904358
2,No log,2.768715,5.322548
3,No log,2.686540,6.561874
4,No log,2.624288,6.870983
5,2.984400,2.576075,5.931671
6,2.984400,2.538190,5.787337
7,2.984400,2.508640,4.894422
8,2.984400,2.482031,4.913330
9,2.650700,2.460144,4.669562
10,2.650700,2.444067,5.518722


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[24660]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[24660]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strate

TrainOutput(global_step=3390, training_loss=1.9916018415692984, metrics={'train_runtime': 2146.996, 'train_samples_per_second': 25.263, 'train_steps_per_second': 1.579, 'total_flos': 804035317727232.0, 'train_loss': 1.9916018415692984, 'epoch': 30.0})

In [22]:
trainer.evaluate()

{'eval_loss': 2.3765249252319336,
 'eval_bleu': 10.878938256764943,
 'eval_runtime': 18.0347,
 'eval_samples_per_second': 11.145,
 'eval_steps_per_second': 0.721,
 'epoch': 30.0}

In [ ]:
trainer.push_to_hub(model_name="eng-mal-translator", commit_message="Model Training complete")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[24660]], 'forced_eos_token_id': 0}


model.safetensors:   0%|          | 0.00/227M [00:00<?, ?B/s]

events.out.tfevents.1720117777.444fcd30b326.928.3:   0%|          | 0.00/7.39k [00:00<?, ?B/s]

events.out.tfevents.1720118500.444fcd30b326.928.4:   0%|          | 0.00/726 [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/Govardhan-06/opus-mt-en-ml/commit/db7a0778893188cc74a45395040a7f82a9befcdd', commit_message='Model Training complete', commit_description='', oid='db7a0778893188cc74a45395040a7f82a9befcdd', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM

model_ckpt = "Govardhan-06/nllb-200-distilled-600M"
translator = pipeline("translation", model=model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

config.json:   0%|          | 0.00/896 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.9k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/4.23k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
user_input=input("Enter the text to be translated: ")
user_input

Enter the text to be translated: Space Exploration Technologies Corporation, commonly referred to as SpaceX, is an American spacecraft manufacturer, launch service provider and satellite communications company headquartered in Hawthorne, California. The company was founded in 2002 by Elon Musk with the goal of reducing space transportation costs and ultimately developing a sustainable colony on Mars. The company currently produces and operates the Falcon 9 and Falcon Heavy rockets along with the Dragon and Starship spacecraft.


'Space Exploration Technologies Corporation, commonly referred to as SpaceX, is an American spacecraft manufacturer, launch service provider and satellite communications company headquartered in Hawthorne, California. The company was founded in 2002 by Elon Musk with the goal of reducing space transportation costs and ultimately developing a sustainable colony on Mars. The company currently produces and operates the Falcon 9 and Falcon Heavy rockets along with the Dragon and Starship spacecraft.'

In [ ]:
input_ids = tokenizer(user_input, return_tensors="pt").input_ids
translated_ids = model.generate(input_ids=input_ids, num_beams=4, early_stopping=True)
translated_text = tokenizer.decode(translated_ids[0], skip_special_tokens=True)
print("Translated Malayalam text:", translated_text)

Translated Malayalam text: സ്പേസ് എക്സ്പ്ലോറേഷൻ ടെക്നോളജീസ് കോർപ്പറേഷൻ, സാധാരണയായി സ്പേസ് എക്സ് എന്ന് വിളിക്കപ്പെടുന്ന, ഒരു അമേരിക്കൻ ബഹിരാകാശ കപ്പൽ നിർമ്മാതാവ്, ലോഞ്ച് സർവീസ് പ്രൊവൈഡർ, കാലിഫോർണിയയിലെ ഹാവോൺ ആസ്ഥാനമായുള്ള സാറ്റലൈറ്റ് കമ്മ്യൂണിക്കേഷൻ കമ്പനിയാണ്. ബഹിരാകാശ ഗതാഗത ചെലവുകൾ കുറയ്ക്കാനും ഒടുവിൽ ചൊവ്വയിൽ സുസ്ഥിരമായ ഒരു കോളനി വികസിപ്പിക്കാനും ലക്ഷ്യമിട്ടാണ് കമ്പനി 2002-ൽ ഇലോൺ മസ്ക് സ്ഥാപിച്ചത്. നിലവിൽ കമ്പനി ഡ്രാഗൺ, സ്റ്റാർ ഷിപ്പ് ബഹിരാകാശ കപ്പുകൾക്കൊപ്പം ഫാൽക്കൺ 9, ഫാൽക്കൺ ഹെവി റോക്കറ്റുകൾ ഉൽപാദിപ്പിക്കുകയും പ്രവർത്തിക്കുകയും ചെയ്യുന്നു.
